In [24]:
import numba as nb
import numpy as np

In [25]:
nb.__version__

'0.46.0'

In [29]:
A_spec = [
    ('a', nb.float64[::100,::100]),
]


@nb.jitclass(A_spec)
class A:
    def __init__(self, _a):
        """
        """

        self.a = _a
        
    def sum2d(self, other):
        if other.flags.f_contiguous:
            raise Exception("Fortran Array Needed")
        return other + self.a

def B_constructor(A_iterable):
    """
    """


    depth = len(A_iterable)
    A_tuple = tuple(A_iterable)

    A_type = nb.deferred_type()
    A_type.define(A.class_type.instance_type)

    B_spec = [
        ('A_tuple', nb.types.UniTuple(A_type, depth)),
    ]

    @nb.jitclass(B_spec)
    class B:
        def __init__(self, _a_tuple):
            """
            """
            self.A_tuple = _a_tuple
            
        def iter(self, init):
            out = np.zeros((10,10))
            for sub_A in self.A_tuple:
                out += sub_A.sum2d(init)
            return np.mean(out)
                
    return B(A_tuple)

In [30]:
num = 2

lst = []
for i in range(num):
    lst.append(A(np.random.rand(100,100)))

B = B_constructor(lst)
B.iter(np.random.rand(100,100))

2.0933488821224913

In [16]:
x = np.random.rand(100,100)

In [23]:
x.flags

  C_CONTIGUOUS : True
  F_CONTIGUOUS : False
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False
  UPDATEIFCOPY : False

In [ ]:
B.iter(12)